<a href="https://colab.research.google.com/github/AnandSinhaProjects/AI_FOR_MEDICAL_TASKS/blob/main/2D/FINE%20TUNING/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
import seaborn as sns
import glob
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, MaxPooling2D, Dense, Dropout, Flatten, Conv2D
from keras.models import Sequential,Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping,LearningRateScheduler

In [ ]:
tdf=pd.read_csv('/content/train.csv')

In [ ]:
tdf.head()

,img_path,class
0,mdb003.pgm.png,dense
1,mdb004.pgm.png,dense
2,mdb033.pgm.png,dense
3,mdb034.pgm.png,dense
4,mdb035.pgm.png,dense


In [ ]:
vdf=pd.read_csv('/content/test.csv')

In [ ]:
vdf.head()

,img_path,class
0,mdb293.pgm.png,fatty
1,mdb297.pgm.png,fatty
2,mdb299.pgm.png,fatty
3,mdb315.pgm.png,dense
4,mdb317.pgm.png,dense


In [ ]:
IMG_SIZE = (224,224)

In [ ]:
t_v = ImageDataGenerator(rescale=1./255.0,
                         height_shift_range=0.1,
                         width_shift_range=0.1,
                         horizontal_flip=True,
                         vertical_flip=False,
                         zoom_range=0.2,
                         rotation_range=20,
                         shear_range=0.1)

In [ ]:
t_f = t_v.flow_from_dataframe(dataframe=tdf,
                              directory=None,
                              x_col ='img_path',
                              y_col='class',
                              class_mode='binary',
                              target_size=IMG_SIZE,
                              batch_size=8)

Found 20 validated image filenames belonging to 2 classes.


In [ ]:
v_v= ImageDataGenerator(rescale=1./255.0)

In [ ]:
v_f = v_v.flow_from_dataframe(dataframe=vdf,
                              directory=None,
                              x_col='img_path',
                              y_col='class',
                              class_mode='binary',
                              target_size=IMG_SIZE,
                              batch_size=8)

Found 6 validated image filenames belonging to 2 classes.


In [ ]:
testX, testY = v_f.next()

In [ ]:
model = VGG16(include_top=True, weights='imagenet')
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input,
                  outputs=transfer_layer.output)

In [ ]:
for layer in vgg_model.layers[0:1]:
    layer.trainable = False

In [ ]:
for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

input_2 False
block1_conv1 True
block1_conv2 True
block1_pool True
block2_conv1 True
block2_conv2 True
block2_pool True
block3_conv1 True
block3_conv2 True
block3_conv3 True
block3_pool True
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [ ]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1, activation='sigmoid'))

In [ ]:

## Set our optimizer, loss function, and learning rate
optimizer = Adam(learning_rate=1e-4)
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']

In [ ]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
new_model.fit_generator(t_f, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
3/3 [==============================] - 38s 15s/step - loss: 0.7784 - binary_accuracy: 0.6500 - val_loss: 0.7045 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 37s 11s/step - loss: 0.6640 - binary_accuracy: 0.6500 - val_loss: 0.6795 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 37s 12s/step - loss: 0.6151 - binary_accuracy: 0.6500 - val_loss: 0.6251 - val_binary_accuracy: 0.3333
Epoch 4/5
3/3 [==============================] - 37s 12s/step - loss: 0.5788 - binary_accuracy: 0.7000 - val_loss: 0.6296 - val_binary_accuracy: 0.6667
Epoch 5/5
3/3 [==============================] - 37s 12s/step - loss: 0.4471 - binary_accuracy: 0.8000 - val_loss: 0.6685 - val_binary_accuracy: 0.6667
